单独测试prompt；
1，对于原始问题，通过更多的问话来引导模型补充完整的内容；
2，对于解决方案，先让模型给出多个思路，然后综合自动生成的与原始的方案，给出一个更加完备的方案。
3，为了避免解决方案过于简单，再对它生成提供测试案例或者更多的问话，以这样的方式来完善其中的技术细节。
4，有了详细的问题描述和解决方案，其他的部分就相对容易了。


In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


load_dotenv("../openai.env")
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0.7, max_tokens=2000)

In [3]:
import pandas as pd

ideas = {}
df = pd.read_excel('patents/rpm-patents-0628d.xlsx')
for i, row in df.iterrows():
    pno, question, solution = row['no.'], row['question'], row['solution']
    if pd.isna(question) or pd.isna(solution): continue
    ideas[pno] = {'question': question, 'solution': solution}

question = ideas['P73']['question']
print('question:', question)
solution = ideas['P73']['solution']
print('solution:', solution)

question: 使用GPT可以生成规模并不复杂的代码，包括RPA代码，一般的方法是设置一个比较详细的生成提示词，但是这样的生成过程难以控制，比如使用了不存在的库函数，或者缺少必要的数据等等，也难以保证生成的流程可以正常执行。
solution: 基本思路是将生成RPA流程拆分为几个步骤，包括需求描述、生成主要步骤、生成每一个步骤的执行代码及测试代码，生成完成之后，先执行测试代码，根据错误提示来更新执行代码，直接所有的测试代码完成。这样，通过多步骤和多次迭代的方式生成可以实际运行的RPA流程。


In [6]:
def build_chain(llm, inputs, outputs, template):
    prompt = PromptTemplate(input_variables=inputs, template=template)
    chain = LLMChain(llm=llm, prompt=prompt, output_key=outputs)
    return chain

In [7]:
template_question = '''你是一个专业的软件方法方向的专利代理，你的任务是帮助专利发明人完善发明点设计。
请根据发明人提供的简要描述和以下规则，重新描述专利发明点要解决的问题。
1，待解决的问题应用于什么样的业务场景？请给出一个简单且有代表性的例子，方便理解。
2，这类问题的一般解决思路是什么？这样的解决思路存在什么样的功能或者性能上的弱点或者缺陷？请给出技术上的理由。
3，为什么要解决这个问题？它有多大的现实价值？请结合业务场景和技术特征进行分析。

待解决问题的简单描述: {raw_question}

请根据以上规则要求，以更完整、更详细的方式重新描述要解决的问题（总长度不超过2000字）：'''

chain = build_chain(llm, ['raw_question'], 'question', template_question)
output = chain.run(question)
print(output)

待解决的问题是如何更准确地使用GPT（生成预训练的Transformer）生成可以正常执行的RPA（机器人流程自动化）代码。这个问题应用于自动化编程的业务场景中，例如，一家软件开发公司希望能够通过GPT来自动生成RPA代码，从而减轻开发人员的工作负担，提高代码生成效率。

一般的解决思路是通过设置一个比较详细的生成提示词，让GPT根据提示词生成代码。然而，这样的解决方案存在一些弱点和缺陷。首先，生成的代码可能会使用到不存在的库函数，这是因为GPT在生成代码时，没有能力去检测库函数是否真实存在，或者是否能够在特定的编程环境下正常工作。其次，生成的代码可能会缺少必要的数据，这是因为GPT在生成代码时，不能像人类开发者那样理解和考虑到代码逻辑中所需要的各种数据。最后，由于这些问题，生成的代码可能无法正常执行，这会导致自动化编程的效果大打折扣。

解决这个问题的现实价值很大。首先，如果能够解决这个问题，就可以大大提高自动化编程的准确性，从而提升代码质量，减少因代码错误而导致的调试时间。其次，解决这个问题也可以提高自动化编程的可靠性，使得开发人员可以更有信心地使用GPT来生成代码。最后，解决这个问题还可以扩大GPT在自动化编程中的应用范围，使得更多的开发任务可以被自动化，从而提高整体的开发效率。

因此，我们需要找到一种方法，可以在GPT生成代码的过程中，检测和修正可能出现的库函数不存在和数据缺失问题，以确保生成的代码可以正常执行，从而提高自动化编程的准确性和可靠性。


In [10]:
template_question2 = '''你是一个专业的软件方法方向的专利代理，你的任务是帮助专利发明人完善发明点设计。
发明人的问题描述往往比较模糊，只集中于核心的概念和逻辑，对于问题相关的背景、价值和创新描述模糊不清。
请根据以下提示，为发明人的简单描述补充相关信息，重新生成一个更好的问题描述：
1，这个发明解决了什么具体的软件或计算问题吗？
2，那些用户、软件系统或计算环境会遇到这个问题？
3，你能举一个这个软件问题通常出现的例子吗？
4，这个问题如何影响软件系统或应用的操作、效率或用户体验？
5，目前存在哪些针对这个问题的软件或算法解决方案？他们在哪些方面不够完善、高效或者对用户不友好？
6，在技术领域或者行业发展中，有哪些最近或即将发生的变化使得这个问题变得特别相关或紧迫？
7，你能量化这个问题吗？有没有任何统计数据或者度量标准能够说明其对计算性能、系统效率或用户体验的影响？
8，这个问题如何影响不同的利益相关者（例如：开发人员、终端用户、系统管理员、业务所有者）？
9，这个软件问题是被广泛认知的，还是说你的发明揭示了它？
10，解决这个问题有没有任何特定的软件、硬件或系统约束？

发明人提供的问题描述: {raw_question}

系统的、详细的、完整的问题描述是：'''

In [11]:
chain = build_chain(llm, ['raw_question'], 'question', template_question2)
output = chain.run(question)
print(output)

该发明针对的是使用生成预训练变换器（GPT）生成规模并不复杂的代码，特别是机器人流程自动化（RPA）代码的问题。在现有的软件开发环境中，开发者经常需要写一些规模并不复杂的代码，包括但不限于RPA代码。然而，当前的方法中，开发者需要设置一个较为详细的生成提示词来引导GPT生成代码，这种方式存在许多问题。

首先，这样的生成过程难以控制，例如，可能会使用不存在的库函数，或者缺少必要的数据等等。举个例子，如果开发者没有在生成提示词中详尽地列出所需的库函数和数据，GPT可能会生成一些包含了不存在的库函数或缺少必要数据的代码，这将导致代码无法正常运行。

其次，该方法难以保证生成的流程可以正常执行。即使生成的代码在语法上没有错误，但由于GPT无法理解代码的逻辑，因此无法保证生成的代码在运行时可以达到预期的效果。比如，代码可能会因为逻辑错误而陷入死循环，或者无法处理某些边界情况。

当前针对这个问题的解决方案主要是人工进行代码检查和调试，但这种方法效率低下，且对开发者的技术水平要求较高。同时，随着人工智能和自动化技术的发展，对于能够自动生成高质量代码的需求越来越强烈。

此外，由于缺乏有效的度量标准，开发者往往难以评价生成代码的质量，也无法系统地改进生成提示词以提高代码质量。这不仅影响了开发效率，也可能导致生成的代码存在潜在的运行风险。

这个问题不仅影响到开发者，也影响到最终用户和业务所有者。例如，如果生成的代码存在问题，可能会导致软件系统运行不稳定，影响用户体验；同时，由于需要花费更多时间进行代码检查和调试，也可能会延迟产品的上市时间，影响业务的运营。

因此，我们需要一种新的方法，能够有效地控制代码生成过程，确保生成的代码可以正常执行，且具有良好的质量。同时，这种方法应该能够适应各种软件、硬件和系统环境，以满足不同的需求。


In [12]:
template_question3 = '''你是一个专业的软件方法方向的专利代理，你的任务是帮助专利发明人完善发明点的问题描述。
发明人提供的问题描述往往非常简单，省略了许多相关的背景、技术、应用场景和价值，而这些对于写专利来说都是必要的内容。
请根据发明人的简单描述，按照以下提示，补充专利问题的相关信息，重新生成一个更好的问题描述：
1，这个发明解决了什么具体的软件或计算问题吗？
2，那些用户、软件系统或业务场景下会遇到这个问题？
3，你能举一个这个软件问题通常出现的例子吗？
4，这个问题如何影响软件系统或应用的操作、效率或用户体验？
5，目前存在哪些针对这个问题的软件或算法解决方案？他们在哪些方面不够完善、高效或者对用户不友好？
6，在技术领域或者行业发展中，有哪些最近或即将发生的变化使得这个问题变得特别相关或紧迫？
7，你能量化这个问题吗？有没有任何统计数据或者度量标准能够说明其对计算性能、系统效率或用户体验的影响？
8，这个问题如何影响不同的利益相关者（例如：开发人员、终端用户、系统管理员、业务所有者）？
9，这个软件问题是被广泛认知的，还是说你的发明揭示了它？
10，解决这个问题有没有任何特定的软件、硬件或系统约束？

发明人提供的问题描述: {raw_question}

重新生成的更系统、更详细、更完整的问题描述：'''

In [13]:
chain = build_chain(llm, ['raw_question'], 'question', template_question3)
output = chain.run(question)
print(output)

此次发明主要解决了使用生成预训练决策树（GPT）生成规模并不复杂的代码，包括机器人流程自动化（RPA）代码时存在的问题。在软件开发和自动化流程设计中，使用GPT可以帮助开发人员快速生成代码，减少手动编写的工作量。然而，目前的方法主要依赖于详细的生成提示词，但这样的生成过程难以进行有效控制。存在的问题包括使用了不存在的库函数，缺少必要的数据等，且难以保证生成的流程可以正常执行。

例如，在实际的软件开发场景中，开发人员可能需要生成某些特定库的函数调用代码。如果生成提示词没有明确指出库函数的限制，GPT可能生成调用了不存在的库函数的代码，这将导致编译错误，影响开发效率。此外，如果生成的代码缺少必要的数据，可能会在运行时导致错误，影响系统的稳定性。

当前的解决方案主要是通过提供更详细的生成提示词来控制GPT的生成过程。但是，这样的方法在精确性和效率上存在问题，不仅需要开发人员花费大量时间精心设计提示词，而且难以保证生成代码的正确性。

随着AI的发展和应用，GPT在代码生成和自动化流程设计中的应用越来越广泛，这使得与之相关的问题变得更为紧迫。据统计，目前使用GPT生成代码的项目中，约有30%的项目存在生成代码错误的问题。

此问题不仅影响开发人员的工作效率，也可能影响终端用户的使用体验。例如，如果生成的RPA代码存在问题，可能导致自动化流程无法正常运行，影响业务的正常进行。

目前，这个问题在开发人员和软件工程学界被广泛关注，本发明旨在提供一种更高效、更可靠的解决方案。解决此问题的过程中需要考虑到GPT模型的特性和限制，以及目标代码的特性和约束。
